# Econ 212: Problem Set 3

In [ ]:
from fredapi import Fred

In [ ]:
fred = Fred(api_key = 'f6fa4f544be19c2ea791e9a3240470bf')

## Problem 1: Seignorage and inflation

## Problem 2: SVAR impulse response to monetary shocks